In [1]:
import time
import stim

from surface_sim import Layout
from surface_sim.setup import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
from surface_sim.layouts import unrot_surface_codes
import surface_sim.experiments.unrot_surface_code_css as exp
from qec_util.distance import get_circuit_distance

from lomatching import MoMatching

# INPUTS
EXPERIMENTS = ["S"]
DISTANCES = [5, 7, 9]
NOISE_MODEL = CircuitNoiseModel
BASES = ["X"]
FRAMES = ["pre-gate", "post-gate"]

This calculation is faster than the other circuit distance of the subgraph because I only perform 2 logical S gates, independently of the code distance.

In [2]:
# GENERATE CIRCUIT
def get_num_qubits(experiment):
    return 1 if experiment in ["I", "S", "H", "SQRT_X"] else 2

def get_circuit(experiment, distance, basis):
    num_qubits = get_num_qubits(experiment)

    # reset
    circuit_str = ""
    if basis == "X":
        circuit_str = "RX " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"
    else:
        circuit_str = "R " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"

    # clifford gates + QEC cycles
    if num_qubits == 1:
        circuit_str += f"{experiment} 0\nTICK\n"*2
    else:
        raise ValueError(f"{experiment} is not known.")

    # measurment
    if basis == "X":
        circuit_str += "MX " + " ".join([f"{i}" for i in range(num_qubits)])
    else:
        circuit_str += "M " + " ".join([f"{i}" for i in range(num_qubits)])

    circuit = stim.Circuit(circuit_str)

    return circuit

# MoMatching
def get_observables(experiment, basis):
    num_qubits = get_num_qubits(experiment)

    if basis == "X" and num_qubits == 1:
        return [["X0"]]
    if basis == "Z" and num_qubits == 1:
        return [["Z0"]]
    if basis == "X" and num_qubits == 2:
        return [["X0"], ["X1"]]
    if basis == "Z" and num_qubits == 2:
        return [["Z0"], ["Z1"]]
    
    raise ValueError


def get_inactive_detectors(dem_subgraph):
    inactive_dets = []
    detector_coords_def = False
    for instr in dem_subgraph.flattened():
        if instr.type == "detector":
            detector_coords_def = True
        if instr.type == "error" and detector_coords_def:
            # the inactive-detector errors appear after the detector-coords definition
            assert instr.args_copy() == [0.5]
            assert len(instr.targets_copy()) == 1
            inactive_dets.append(instr.targets_copy()[0].val)
            
    return inactive_dets

def remove_inactive_detectors_from_circuit(circuit, inactive_dets):
    new_circuit = stim.Circuit()
    det_id = 0
    for instr in circuit.flattened():
        if instr.name != "DETECTOR":
            new_circuit.append(instr)
            continue

        if det_id not in inactive_dets:
            new_circuit.append(instr)
            
        det_id += 1

    return new_circuit

def remove_inactive_logicals_from_circuit(circuit, active_logs):
    new_circuit = stim.Circuit()
    log_id = 0
    num_logs = 0
    for instr in circuit.flattened():
        if instr.name != "OBSERVABLE_INCLUDE":
            new_circuit.append(instr)
            continue

        if log_id in active_logs:
            # rewirte the logical so that the SAT solver does not complain
            new_instr = stim.CircuitInstruction(name="OBSERVABLE_INCLUDE", gate_args=[num_logs], targets=instr.targets_copy())
            new_circuit.append(new_instr)
            num_logs += 1
            
        log_id += 1

    return new_circuit

In [3]:
for experiment_name in EXPERIMENTS:
    for basis in BASES:
        for frame in FRAMES:
            for distance in DISTANCES:
                layouts = unrot_surface_codes(get_num_qubits(experiment_name), distance=distance)
                qubit_inds = {}
                anc_coords = {}
                anc_qubits = []
                stab_coords = {}
                for l, layout in enumerate(layouts):
                    qubit_inds.update(layout.qubit_inds)
                    anc_qubits += layout.get_qubits(role="anc")
                    coords = layout.anc_coords
                    anc_coords.update(coords)
                    stab_coords[f"Z{l}"] = [v for k,v in coords.items() if k[0] == "Z"]
                    stab_coords[f"X{l}"] = [v for k,v in coords.items() if k[0] == "X"]

                circuit = get_circuit(experiment_name, distance, basis)

                setup = CircuitNoiseSetup()
                setup.set_var_param("prob", 1e-3)
                model = NOISE_MODEL(setup=setup, qubit_inds=qubit_inds)
                detectors = Detectors(anc_qubits, frame=frame, anc_coords=anc_coords)

                schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)
                experiment = experiment_from_schedule(
                    schedule, model, detectors, anc_reset=True, anc_detectors=None
                )
                
                dem = experiment.detector_error_model(allow_gauge_detectors=False)
                decoder = MoMatching(dem, circuit, get_observables(experiment_name, basis), stab_coords, frame)

                print("start=", time.strftime("%D %H:%M:%S"))
                d_circ_list = []
                for i in range(get_num_qubits(experiment_name)):
                    inactive_dets = get_inactive_detectors(decoder.decoding_subgraphs[i])
                    experiment_subgraph = remove_inactive_detectors_from_circuit(experiment, inactive_dets=inactive_dets)
                    experiment_subgraph = remove_inactive_logicals_from_circuit(experiment_subgraph, active_logs=[i])
                    d_circ = get_circuit_distance(experiment_subgraph)
                    d_circ_list.append(d_circ)
                print("finish=", time.strftime("%D %H:%M:%S"))
                    
                print(f"{experiment_name} b={basis} f={frame} d={distance} d_circ_subgraph={d_circ_list}")

start= 04/28/25 23:36:18
finish= 04/28/25 23:36:21
S b=X f=pre-gate d=5 d_circ_subgraph=[4]
start= 04/28/25 23:36:22
finish= 04/29/25 00:32:31
S b=X f=pre-gate d=7 d_circ_subgraph=[6]
start= 04/29/25 00:32:32


/Users/marcserraperal/virtual_environments/log_s/lib/python3.12/site-packages/pysat/examples/rc2.py:1738: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\.wcnf[p|+]?(\.(gz|bz2|lzma|xz))?$', files[0]):


error: Caught keyboard interrupt

In [ ]:
# it ran for 30h without finding the subgraph circuit distance